In [74]:
#####Modelling Reciprocating Relationships with Hawkes Processes#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [75]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z

In [76]:
####データの発生####
##データの設定
k1 = 10
k2 = 8
hh = 5000
item = 3000
pt = np.random.poisson(np.random.gamma(10.0, 1/0.25, hh), hh); pt[pt <= 5] = 5
hhpt = np.sum(pt)

In [77]:
##IDとインデックスを設定
#IDを設定
d_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスを設定
d_list = [i for i in range(hh)]
d_vec = [i for i in range(hh)]
pt_list = [j for j in range(np.max(pt))]
pt_n = np.repeat(0, np.max(pt))
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")
    d_vec[i] = np.repeat(1, pt[i])
for j in range(np.max(pt)):
    pt_list[j] = np.array(np.where(pt_id==j)[0], dtype="int")
    pt_n[j] = pt_list[j].shape[0]
max_index = np.array([np.max(d_list[i]) for i in range(hh)])

In [78]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(hhpt, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[d_list[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))
    
#インデックスの設定
item_list = [j for j in range(item)]
item_vec = [j for j in range(item)]
item_n = np.repeat(0, item)
for j in range(item):
    item_list[j] = np.array(np.where(item_id==j), dtype="int")
    item_vec[j] = np.repeat(1, len(item_list[j]))
    item_n[j] = len(item_list[j])

0
1000
2000
3000
4000


In [79]:
##アイテム購買履歴を設定
#データの設定
max_pt = np.max(pt)
item_dt_list = [i for i in range(hh)]

#アイテムごとに購買履歴を格納
for i in range(hh):
    temp_dt = np.zeros((pt[i], max_pt), dtype="int16")
    temp_id = item_id[d_list[i]] + 1
    temp_index = np.arange(pt[i])
    for j in range(pt[i]):
        index = np.append(np.arange(j, pt[i]), np.arange(0, j))
        temp_dt[j, temp_index] = temp_id[index] 
    temp_dt = np.tril(temp_dt)
    item_dt_list[i] = temp_dt
    
#リストを変換
item_dt = np.array(list(itertools.chain(*[item_dt_list[i] for i in range(hh)]))) - 1
del item_dt_list

#インデックスの設定
dt_list = [j for j in range(max_pt)]
dt_n = np.repeat(0, max_pt)
for j in range(max_pt):
    dt_list[j] = np.array(np.where(item_dt[:, j]!=-1)[0], dtype="int")
    dt_n[j] = dt_list[j].shape[0]

In [80]:
##応答変数が妥当な値になるまで繰り返す
rp = 0
while True:
    rp = rp + 1

    ##パラメータを生成
    #潜在変数を生成
    theta1 = np.random.dirichlet(np.repeat(2.0, k1), 1).reshape(-1)
    theta2 = np.random.dirichlet(np.repeat(2.0, k2), 1).reshape(-1)
    Z1 = np.random.multinomial(1, theta1, hh)
    Z2 = np.random.multinomial(1, theta2, item)
    z1_vec = np.dot(Z1, np.arange(k1)); z2_vec = np.dot(Z2, np.arange(k2))

    #ガンマ分布の事前分布のパラメータ
    er1 = 3.0; er2 = 2.0; er3 = 0.3
    alpha1 = np.full((k1, k2), er1); beta1 = 1.5
    alpha2 = np.full((k1, k2), er2); beta2 = 2.0
    alpha3 = np.full((k1, k2), er3); beta3 = 2.0

    #モデルパラメータを生成
    theta1 = np.random.gamma(alpha1, 1/beta1).reshape(k1, k2)
    theta2 = np.random.gamma(alpha2, 1/beta2).reshape(k1, k2)
    theta3 = np.random.gamma(alpha3, 1/beta3).reshape(k1, k2)
    thetat1 = theta1.copy(); thetat2 = theta2.copy(); thetat3 = theta3.copy()
    
    ##応答変数を生成
    #データの設定
    trunc = 100
    y = np.repeat(0.0, hhpt)
    y_cumulative = np.repeat(0.0, hhpt)
    y_dt_list1 = [i for i in range(hh)]
    y_dt_list2 = [i for i in range(hh)]
    
    #ユーザーごとにデータを生成
    for i in range(hh):
        #データの格納用配列
        index = d_list[i]
        temp_index = np.arange(pt[i])
        temp_z1 = z1_vec[d_id[temp_index]]
        temp_z2 = Z2[item_id[temp_index], ]
        temp_dt1 = np.zeros((pt[i], max_pt))
        temp_dt2 = np.zeros((pt[i], max_pt))
        temp_y1 = np.repeat(0.0, pt[i])
        temp_y2 = np.repeat(0.0, pt[i])

        for j in range(pt[i]):
            if j==0:
                #期待値を定義
                Lambda = np.sum(theta1[temp_z1[j], ] * temp_z2[j, ])
                
                #指数分布から応答変数を生成
                temp_y1[j] = np.random.exponential(Lambda, 1)
                temp_y2[j] = temp_y1[j]
            else:
                #期待値を定義
                Lambda1 = np.sum(theta1[temp_z1[j], ] * temp_z2[j, ])
                Lambda2 = np.sum(theta2[temp_z1[j-1], ] * temp_z2[j-1, ])
                Lambda3 = np.sum(theta3[temp_z1[1:j-1], ] * temp_z2[1:j-1, ], axis=1) * (temp_dt2[j, j] - temp_dt2[j, 1:j-1]) / trunc
                Lambda = Lambda1 + Lambda2 + np.sum(Lambda3)
                
                #指数分布から応答変数を生成
                temp_y1[j] = np.random.exponential(Lambda, 1)
                temp_y2[j] = temp_y1[j]
                temp_dt1[np.arange(j, pt[i]), j] = np.repeat(temp_y1[j-1], pt[i]-j)
                temp_dt2 = np.cumsum(temp_dt1, axis=1)
                
        #データを格納
        y[index] = temp_y1
        y_cumulative[index] = temp_y2
        y_dt_list1[i] = np.tril(temp_dt1)
        y_dt_list2[i] = np.tril(temp_dt2)
        
    #break条件
    print([rp, np.max(y_cumulative), np.min(y_cumulative[max_index])])
    if (np.max(y_cumulative) < 500) & (np.max(y_cumulative) > 150):
        break

[1, 28809.451869991524, 0.0021197565991286902]
[2, 16260.818961385245, 0.0009505289346337562]
[3, 12251.176544424636, 0.0022427688785858274]
[4, 10338.899846075794, 0.003328123673321297]
[5, 6393.69803917076, 0.0017866716603445406]
[6, 1454.535340763148, 0.0011565400885942994]
[7, 205868.47564548458, 0.003333138852144348]
[8, 232.41948617256415, 0.0008751212299953277]


In [83]:
#リストを変換
y_dt1 = np.array(list(itertools.chain(*[y_dt_list1[i] for i in range(hh)]))) 
y_dt2 = np.array(list(itertools.chain(*[y_dt_list2[i] for i in range(hh)]))) 
del y_dt_list1, y_dt_list2

In [84]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##切断指数分布の乱数を生成する関数
def rtexp(gamma, a, b):
    #切断指数分布の乱数を生成
    FA = scipy.stats.expon.cdf(a, scale=gamma)
    FB = scipy.stats.expon.cdf(b, scale=gamma)
    par = scipy.stats.expon.ppf(np.random.uniform(0, 1, a.shape[0])*(FB-FA)+FA, scale=gamma)
    return par

In [85]:
##データの設定
R = 2000
keep = 4
burnin = int(500/keep)
iter = 0
disp = 10
e1 = 0.001
e2 = 0.0025
L = 3

In [86]:
##事前分布の設定
alpha1 = 1.0; alpha2 = 1.0
beta1 = 1.0; beta2 = 1.0

In [90]:
##パラメータの真値
#潜在変数の真値
Zi1 = Z1.copy()
Zi2 = Z2.copy()
z1_vec = np.dot(Zi1, np.arange(k1))
z2_vec = np.dot(Zi2, np.arange(k2))

#モデルパラメータの真値
theta1 = thetat1.copy()
theta2 = thetat2.copy()

In [88]:
##パラメータの初期値
#潜在変数の初期値
Zi1 = np.random.multinomial(1, np.repeat(1/k1, k1), hh)
Zi2 = np.random.multinomial(1, np.repeat(1/k2, k2), item)
z1_vec = np.dot(Zi1, np.arange(k1))
z2_vec = np.dot(Zi2, np.arange(k2))

#モデルパラメータの初期値
theta1 = np.random.gamma(alpha1, 1/beta1, k1*k2).reshape(k1, k2)
theta2 = np.random.gamma(alpha2, 1/beta2, k1*k2).reshape(k1, k2)

In [92]:
####ギブスサンプリングでパラメータをサンプリング####

##潜在変数をサンプリング



array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.87412169,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.87412169,  5.06367023, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  1.15749528, 13.23537734, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.15749528, 13.23537734, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.15749528, 13.23537734, ...,  0.        ,
         0.        ,  0.        ]])